# Economic Data Analysis

We are going to use the [FRED API](https://fred.stlouisfed.org/docs/api/fred/) to get the data

In [8]:
# import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from fredapi import Fred

# set styles
plt.style.use('fivethirtyeight')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
# pd.options.display.max_rows = 500

# set up fred api
fred_api_key = os.environ.get('FRED_API_KEY')
# Create a fred object
fred = Fred(api_key=fred_api_key)

Search for a topic in Fred

In [10]:
# search for economic data in fred
fred.search('S&P 500')[:5]

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
SP500,SP500,2023-01-22,2023-01-22,S&P 500,2013-01-22,2023-01-20,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-01-20 19:10:57-06:00,84,The observations for the S&P 500 represent the...
VXVCLS,VXVCLS,2023-01-22,2023-01-22,CBOE S&P 500 3-Month Volatility Index,2007-12-04,2023-01-19,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-01-20 08:36:08-06:00,55,"Copyright, 2016, Chicago Board Options Exchang..."
MEHOINUSA672N,MEHOINUSA672N,2023-01-22,2023-01-22,Real Median Household Income in the United States,1984-01-01,2021-01-01,Annual,A,2021 CPI-U-RS Adjusted Dollars,2021 CPI-U-RS Adjusted $,Not Seasonally Adjusted,NSA,2022-09-13 13:48:01-05:00,80,Household data are collected as of March. As ...
STLFSI,STLFSI,2023-01-22,2023-01-22,St. Louis Fed Financial Stress Index (DISCONTI...,1993-12-31,2020-03-13,"Weekly, Ending Friday",W,Index,Index,Not Seasonally Adjusted,NSA,2020-03-19 09:00:02-05:00,44,The methodology for the St. Louis Fed's Financ...
MEHOINUSA646N,MEHOINUSA646N,2023-01-22,2023-01-22,Median Household Income in the United States,1984-01-01,2021-01-01,Annual,A,Current Dollars,Current $,Not Seasonally Adjusted,NSA,2022-09-13 13:48:03-05:00,62,Household data are collected as of March. As ...


In [5]:
fred